In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
data = data.sample(10000)
data.head()

,review,sentiment
22892,"Labeling this film a ""lesbian love story"" is a...",positive
9148,This oddity from Roman Polanski clearly shows ...,positive
25570,Story of a good-for-nothing poet and a sidekic...,negative
27081,I remember watching this film back in 86' when...,negative
15497,Leland P. Fitzgerald (Ryan Gosling) has commit...,positive


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 22892 to 28466
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [4]:
data['sentiment'].replace('positive',1,inplace=True)
data['sentiment'].replace('negative',0,inplace=True)
data.head()

/tmp/ipykernel_35/3093252986.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['sentiment'].replace('positive',1,inplace=True)
/tmp/ipykernel_35/3093252986.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

,review,sentiment
22892,"Labeling this film a ""lesbian love story"" is a...",1
9148,This oddity from Roman Polanski clearly shows ...,1
25570,Story of a good-for-nothing poet and a sidekic...,0
27081,I remember watching this film back in 86' when...,0
15497,Leland P. Fitzgerald (Ryan Gosling) has commit...,1


In [5]:
import re
def remove_html(text):
  clean = re.compile("<.*?>")
  return re.sub(clean, '', text)
    

In [6]:
data['review'] = data['review'].apply(remove_html)
data.head()

,review,sentiment
22892,"Labeling this film a ""lesbian love story"" is a...",1
9148,This oddity from Roman Polanski clearly shows ...,1
25570,Story of a good-for-nothing poet and a sidekic...,0
27081,I remember watching this film back in 86' when...,0
15497,Leland P. Fitzgerald (Ryan Gosling) has commit...,1


In [7]:
def convert_lower(text):
  return text.lower()

In [8]:
data['review'] = data['review'].apply(convert_lower)
data.head()

,review,sentiment
22892,"labeling this film a ""lesbian love story"" is a...",1
9148,this oddity from roman polanski clearly shows ...,1
25570,story of a good-for-nothing poet and a sidekic...,0
27081,i remember watching this film back in 86' when...,0
15497,leland p. fitzgerald (ryan gosling) has commit...,1


In [9]:
def remove_special(text):
  x = ''
  for i in text:
    if i.isalnum():
      x+=i
    else:
      x += ' '
  return x

In [10]:
data['review'] = data['review'].apply(remove_special)
data.head()

,review,sentiment
22892,labeling this film a lesbian love story is a...,1
9148,this oddity from roman polanski clearly shows ...,1
25570,story of a good for nothing poet and a sidekic...,0
27081,i remember watching this film back in 86 when...,0
15497,leland p fitzgerald ryan gosling has commit...,1


In [11]:
import nltk
from nltk.corpus import stopwords

In [12]:
def remove_stopwords(text):
  x = []
  for i in text.split():
    if i not in stopwords.words("english"):
      x.append(i)
  y = x[:]
  x.clear()
  return y

In [13]:
data['review'] = data['review'].apply(remove_stopwords)
data.head()

,review,sentiment
22892,"[labeling, film, lesbian, love, story, accurat...",1
9148,"[oddity, roman, polanski, clearly, shows, preo...",1
25570,"[story, good, nothing, poet, sidekick, singer,...",0
27081,"[remember, watching, film, back, 86, first, ca...",0
15497,"[leland, p, fitzgerald, ryan, gosling, committ...",1


In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [15]:
def stem_words(text):
  y = []
  for i in text:
    y.append(ps.stem(i))
  z = y[:]
  y.clear()
  return z

In [16]:
data['review'] = data['review'].apply(stem_words)
data.head()

,review,sentiment
22892,"[label, film, lesbian, love, stori, accur, cal...",1
9148,"[odditi, roman, polanski, clearli, show, preoc...",1
25570,"[stori, good, noth, poet, sidekick, singer, pu...",0
27081,"[rememb, watch, film, back, 86, first, came, a...",0
15497,"[leland, p, fitzgerald, ryan, gosl, commit, un...",1


In [17]:
def join_words(input_list):
  return " ".join(input_list)

In [18]:
data['review'] = data['review'].apply(join_words)
data.head()

,review,sentiment
22892,label film lesbian love stori accur call pride...,1
9148,odditi roman polanski clearli show preoccup la...,1
25570,stori good noth poet sidekick singer put word ...,0
27081,rememb watch film back 86 first came aw film a...,0
15497,leland p fitzgerald ryan gosl commit unspeak c...,1


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [20]:
x = cv.fit_transform(data['review']).toarray()

In [21]:
x.shape

(10000, 36128)

In [22]:
y = data.iloc[:,-1].values
y

array([1, 1, 0, ..., 1, 0, 0])

In [23]:
y.shape

(10000,)

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [25]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [26]:
model1 = GaussianNB()
model2 = MultinomialNB()
model3 = BernoulliNB()

In [30]:
y_train.shape

(8000,)

In [31]:

model1.fit(x_train, y_train)
model2.fit(x_train, y_train)
model3.fit(x_train, y_train)

BernoulliNB()

In [33]:
y_pred1 = model1.predict(x_test)
y_pred2 = model2.predict(x_test)
y_pred3 = model3.predict(x_test)

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
print("Guassian:", accuracy_score(y_test, y_pred1))
print("Multinomial:", accuracy_score(y_test, y_pred2))
print("Bernoulli:", accuracy_score(y_test, y_pred3))

Guassian: 0.6145
Multinomial: 0.8425
Bernoulli: 0.8385
